In [5]:
import torch
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


In [6]:
eval_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

test_dir = "../dataset/test"

test_dataset = datasets.ImageFolder(test_dir, transform=eval_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [11]:
class CarClassifier(nn.Module):
    def __init__(self, backbone, num_classes):
        super().__init__()
        self.backbone = backbone
        self.classifier = nn.Sequential(
            nn.BatchNorm1d(2048),
            nn.Dropout(p=0.5),
            nn.Linear(2048, num_classes)
        )

    def forward(self, x):
        feats = self.backbone(x)
        return self.classifier(feats)


In [ ]:
num_classes = len(train_dataset.classes)
print("num_classes:", num_classes)


In [8]:
model = CarClassifier(backbone, num_classes)

state = torch.load(
    "saved_models/resnet50_compcars_20260111_154612.pth",
    map_location=device
)

model.load_state_dict(state)
model.to(device)
model.eval()


CarClassifier(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
   

In [10]:
import torch
import torch.nn as nn
from torchvision.models import resnet50
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [15]:
# ===============================
# 1. Imports
# ===============================
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torchvision.models import resnet50
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# ===============================
# 2. Device
# ===============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ===============================
# 3. Paths (ADJUST ONLY IF NEEDED)
# ===============================
DATASET_ROOT = "../dataset"
MODEL_PATH = "saved_models/resnet50_compcars_20260111_154612.pth"

# ===============================
# 4. Transforms (MATCH eval_transform)
# ===============================
eval_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# ===============================
# 5. Datasets & Loaders
# ===============================
train_dataset = datasets.ImageFolder(f"{DATASET_ROOT}/train", transform=eval_transform)
val_dataset   = datasets.ImageFolder(f"{DATASET_ROOT}/val",   transform=eval_transform)
test_dataset  = datasets.ImageFolder(f"{DATASET_ROOT}/test",  transform=eval_transform)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=4
)

num_classes = len(train_dataset.classes)
print("num_classes:", num_classes)

# ===============================
# 6. Model definition (EXACT match)
# ===============================
class CarClassifier(nn.Module):
    def __init__(self, backbone, num_classes):
        super().__init__()
        self.backbone = backbone
        self.classifier = nn.Sequential(
            nn.BatchNorm1d(2048),
            nn.Dropout(0.5),
            nn.Linear(2048, num_classes)
        )

    def forward(self, x):
        feats = self.backbone(x)
        return self.classifier(feats)

# ===============================
# 7. Recreate backbone + load weights
# ===============================
backbone = resnet50(weights=None)
backbone.fc = nn.Identity()

model = CarClassifier(backbone, num_classes)

state = torch.load(MODEL_PATH, map_location=device)
model.load_state_dict(state)

model.to(device).eval()

print("Model loaded successfully")


Using device: cpu
num_classes: 429
Model loaded successfully


In [16]:
def evaluate(model, loader):
    y_true, y_pred = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Evaluating"):
            imgs = imgs.to(device)
            labels = labels.to(device)

            logits = model(imgs)
            preds = logits.argmax(1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision_macro": precision_score(y_true, y_pred, average="macro", zero_division=0),
        "recall_macro": recall_score(y_true, y_pred, average="macro", zero_division=0),
        "f1_macro": f1_score(y_true, y_pred, average="macro", zero_division=0),
    }

metrics = evaluate(model, test_loader)
metrics


Evaluating: 100%|██████████| 467/467 [09:29<00:00,  1.22s/it]


{'accuracy': 0.6983371328952662,
 'precision_macro': 0.7458973063632867,
 'recall_macro': 0.6847076660729721,
 'f1_macro': 0.6976176883172942}